<a href="https://colab.research.google.com/github/hy30n80/Data-Structure-/blob/main/06_Abstract_data_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EC2202 Abstract Data Types

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/mVc5V68wIV4" title="YouTube video player" frameborder="0" allowfullscreen></iframe>

**Disclaimer.**
This code examples are based on

1. [KAIST CS206 (Professor Otfried Cheong)](https://otfried.org/courses/cs206/)

## First Implementation

Let's first implement our date calculator using the Julian day convention.

[**The Julian day**](https://en.wikipedia.org/wiki/Julian_day) is the continuous count of days since the beginning of the Julian period, and is used primarily by astronomers, and in software for easily calculating elapsed days between two events (e.g. food production date and sell by date). The Julian day number (JDN) is the integer assigned to a whole solar day in the Julian day count starting from noon Universal Time, with Julian day number 0 assigned to the day starting at noon on Monday, January 1, 4713 BC, proleptic Julian calendar (November 24, 4714 BC, in the proleptic Gregorian calendar), a date at which three multi-year cycles started (which are: Indiction, Solar, and Lunar cycles) and which preceded any dates in recorded history. For example, the Julian day number for the day starting at 12:00 UT (noon) on January 1, 2000, was 2 451 545.

In [1]:
class Date():
  def __init__(self, year, month, day):
    # encapsulation
    self._year = year
    self._month = month
    self._day = day

  def year(self):
    return self._year

  def month(self):
    return self._month

  def day(self):
    return self._day

  def day_of_week(self):
    jday = self._to_jday()
    return jday % 7

  def _to_jday(self):
    tmp = 0
    if self._month < 3:
      tmp = -1
    return (self._day - 32075 +
           (1461 * (self._year + 4800 + tmp) // 4) +
           (367 * (self._month - 2 - tmp * 12) // 12) -
           (3 * ((self._year + 4900 + tmp) // 100) // 4))

  @staticmethod  # decorator
  def _jday_to_ymd(jday):
    A = jday + 68569
    B = 4 * A // 146097
    A = A - (146097 * B + 3) // 4
    year = 4000 * (A + 1) // 1461001
    A = A - (1461 * year // 4) + 31
    month = 80 * A // 2447
    day = A - (2447 * month // 80)
    A = month // 11
    month = month + 2 - (12 * A)
    year = 100 * (B - 49) + year + A
    return year, month, day

  def num_days(self, other_date):
    return other_date._to_jday() - self._to_jday()

  def is_leap_year(self):
    return ((self._year % 400 == 0) or
            ((self._year % 4 == 0) and (self._year % 100 != 0)))

  def __str__(self):
    return "%04d/%02d/%02d" % (self._year, self._month, self._day)

  def advance_by(self, days):
    jday = self._to_jday() + days
    y, m, d = self._jday_to_ymd(jday)
    return Date(y, m, d)

Now, let's test our implementation

In [ ]:
month_names = [ "January", "February", "March", "April",
               "May", "June", "July", "August",
               "September", "October", "November", "December" ]
day_names = [ "Monday", "Tuesday", "Wednesday", "Thursday",
             "Friday", "Saturday", "Sunday" ]
digits = "0123456789"

def str_to_ymd(s):
  if len(s) != 10 or s[4] not in "/.-" or s[7] != s[4]:
    return 0, 0, 0
  for i in range(10):
    if i != 4 and i != 7 and s[i] not in digits:
      return 0, 0, 0
  year = int(s[:4])
  month = int(s[5:7])
  day = int(s[8:])
  return year, month, day

def get_date(s):
  y, m, d = str_to_ymd(s)
  if y > 0:
    return Date(y, m, d)
  print("Incorrect date format")
  return None

def show_weekday(s):
  day = get_date(s)
  if day:
    print(day, "is a", day_names[day.day_of_week()])

def show_difference(s1, s2):
  day1 = get_date(s1)
  if day1:
    day2 = get_date(s2)
    if day2:
      print("There are", day1.num_days(day2), "days between", day1, "and", day2)

def show_advance(s1, op, s2):
  day = get_date(s1)
  if day:
    n = int(s2)
    if op not in "+-":
      print("Incorrect operator (only plus and minus are possible)")
      return
    m = n
    if op == "-":
      m = -n
    print(day, op, n, "days =", day.advance_by(m))

def main():
  print("Welcome to the EC2202 day calculator\n")
  print("You can do the following:")
  print(" * Enter a date to determine the weekday")
  print(" * Enter two dates to determine the number of days in between")
  print(" * Enter a date, + or -, and then a number of days\n")
  print("All dates are in the form YYYY/MM/DD\n")

  while True:
    s = input("> ")
    f = s.split()
    if len(f) == 0:
      return
    elif len(f) == 1:
      show_weekday(f[0])
    elif len(f) == 2:
      show_difference(f[0], f[1])
    elif len(f) == 3:
      show_advance(f[0], f[1], f[2])
    else:
      print("Incorrect command")

main()

Welcome to the EC2202 day calculator

You can do the following:
 * Enter a date to determine the weekday
 * Enter two dates to determine the number of days in between
 * Enter a date, + or -, and then a number of days

All dates are in the form YYYY/MM/DD

> 2022/04/14 - 30
2022/04/14 - 30 days = 2022/03/15
> 2022/03/21 1990/04/14
There are -11664 days between 2022/03/21 and 1990/04/14
> 


## Alternative Implementation

Consider a situation where you need to store millions of `Date` objects. In this case it would be best to make the `Date` object as small as possible. Instead of storing three integers for year, month, and day, we could just store a single integer—the Julian day number.

In [ ]:
def _jday_to_ymd(jday):
  A = jday + 68569
  B = 4 * A // 146097
  A = A - (146097 * B + 3) // 4
  year = 4000 * (A + 1) // 1461001
  A = A - (1461 * year // 4) + 31
  month = 80 * A // 2447
  day = A - (2447 * month // 80)
  A = month // 11
  month = month + 2 - (12 * A)
  year = 100 * (B - 49) + year + A
  return year, month, day

def _to_jday(year, month, day):
  tmp = 0
  if month < 3:
    tmp = -1
  return (day - 32075 +
          (1461 * (year + 4800 + tmp) // 4) +
          (367 * (month - 2 - tmp * 12) // 12) -
          (3 * ((year + 4900 + tmp) // 100) // 4))

class Date():
  def __init__(self, year, month, day):
    self._jday = _to_jday(year, month, day)

  # ppp exercise
  def _to_ymd(self):
    pass

  def year(self):
    pass

  def month(self):
    pass

  def day(self):
    pass

  def __str__(self):
    y, m, d = self._to_ymd()
    return "%04d/%02d/%02d" % (y, m, d)

  def day_of_week(self):
    pass

  def num_days(self, other_date):
    pass

  def is_leap_year(self):
    y = self.year()
    return ((y % 400 == 0) or
            ((y % 4 == 0) and (y % 100 != 0)))

  def advance_by(self, days):
    pass

## Comparisons

We have not yet added the comparison operations to our implementations. It’s slightly easier doing this for the second implementation, since we can just compare the Julian day numbers. It suffices to define a few magic methods:


In [ ]:
def _jday_to_ymd(jday):
  A = jday + 68569
  B = 4 * A // 146097
  A = A - (146097 * B + 3) // 4
  year = 4000 * (A + 1) // 1461001
  A = A - (1461 * year // 4) + 31
  month = 80 * A // 2447
  day = A - (2447 * month // 80)
  A = month // 11
  month = month + 2 - (12 * A)
  year = 100 * (B - 49) + year + A
  return year, month, day

def _to_jday(year, month, day):
  tmp = 0
  if month < 3:
    tmp = -1
  return (day - 32075 +
          (1461 * (year + 4800 + tmp) // 4) +
          (367 * (month - 2 - tmp * 12) // 12) -
          (3 * ((year + 4900 + tmp) // 100) // 4))

class Date():
  def __init__(self, year, month, day):
    self._jday = _to_jday(year, month, day)

  def _to_ymd(self):
    return _jday_to_ymd(self._jday)

  def year(self):
    return self._to_ymd()[0]

  def month(self):
    return self._to_ymd()[1]

  def day(self):
    return self._to_ymd()[2]

  def __str__(self):
    y, m, d = self._to_ymd()
    return "%04d/%02d/%02d" % (y, m, d)

  def day_of_week(self):
    return self._jday % 7

  def num_days(self, other_date):
    return other_date._jday - self._jday

  def is_leap_year(self):
    y = self.year()
    return ((y % 400 == 0) or
            ((y % 4 == 0) and (y % 100 != 0)))

  def advance_by(self, days):
    y, m, d = _jday_to_ymd(self._jday + days)
    return Date(y, m, d)

  def __eq__(self, rhs):
    return self._jday == rhs._jday

  def __lt__(self, rhs):
    return self._jday < rhs._jday

  def __le__(self, rhs):
    return self._jday <= rhs._jday

The `__eq__` method implements equality, and makes the == and != operators work correctly. The `__lt__` method implements the “less than” operator <, while the `__le__` method implements the “less than or equal” operator <=. Python is smart enough to also use these methods for the > and >= operators.

## Exceptions

Neither of our Date implementations handled invalid dates in the constructor:

In [ ]:
# from date2 import Date

d = Date(2017, 8, 32)

We would like to print an error message when the user enters such a date, but our Date data type accepts it silently.
It’s actually quite easy to detect invalid dates: we can simply convert year, month, and day to the Julian day number, then convert it back and check if we get the same values.
The problem is that the constructor always returns a Date object—so how can we report that the arguments are invalid? The solution is to raise an exception:

In [ ]:
def __init__(self, year, month, day):
  jday = _to_jday(year, month, day)
  y, m, d = _jday_to_ymd(jday)
  if y != year or m != month or d != day:
    raise ValueError("Invalid Gregorian date")
  self._jday = jday

Let's test again :)

In [ ]:
# from date4 import Date
d = Date(2017, 8, 32)

now we also need to handle the exception in our client code, otherwise the program will simply crash when the user enters an invalid date.

In [ ]:
def main():
  print("Welcome to the EC2202 day calculator\n")
  print("You can do the following:")
  print(" * Enter a date to determine the weekday")
  print(" * Enter two dates to determine the number of days in between")
  print(" * Enter a date, + or -, and then a number of days\n")
  print("All dates are in the form YYYY/MM/DD\n")
  while True:
    s = input("> ")
    f = s.split()
    try:
      if len(f) == 0:
        return
      elif len(f) == 1:
        show_weekday(f[0])
      elif len(f) == 2:
        show_difference(f[0], f[1])
      elif len(f) == 3:
        show_advance(f[0], f[1], f[2])
      else:
        print("Incorrect command")
    except ValueError as e:
      print(e)

main()

## More Exceptions

You may have seen many times how your programs terminate with an error or exception message. Here are some examples of exception messages:

In [2]:
a = 3
a // 0

ZeroDivisionError: integer division or modulo by zero

In [3]:
s = 'abc'
b = int(s)

ValueError: invalid literal for int() with base 10: 'abc'

In [ ]:
data = [ None ] * 10000000000

Some exceptions, such as `MemoryError`, indicate a serious failure, where continuing the program makes no sense.

Other exceptions, however, merely indicate an unexpected or abnormal condition in a program. For instance, a mistake in the input data of a program could cause an exception. Such mistakes can be handled: We say that the exception is handled or caught.

We can catch the exception by enclosing the critical part in a `try`-block, and adding an `except`-clause to handle the exceptions we are interested in:

In [ ]:
s = input("Enter an integer> ")
try:
  x = int(s)
  print("You said: %g" % x)
except ValueError:
  print("’%s’ is not a number" % s)

If the `try`-block executes normally, then the `except`-clauses are skipped. But if somewhere inside the `try`-block (including in any method called, directly or indirectly) an exception is thrown, then execution of the `try`-block stops immediately, and continues in the first `except`-clause that matches the exception. Here, “matches” means that the exception is the same type as the exception type listed in the clause.
The code within an `except`-clause is called an exception handler.

The nice thing about exceptions is that you can also catch exceptions that were thrown inside functions called in the `try`-block.

In [ ]:
def test(s):
  return int(100 * float(s))

def show(s):
  try:
    print(test(s))
  except ValueError:
    print("’%s’ is not a number" % s)

In [ ]:
show("123.456")

In [ ]:
show("123a456")

Note that when an exception occurs (in this case inside the `float` function), the `float` function does not return, the `test` function does not return, the print statement is not executed, and instead execution continues in the `except`-clause.

In [ ]:
def f(n):
  print("Starting f(%d) ... " % n)
  g(n)
  print("Ending f(%d) ... " % n)

def g(n):
  print("Starting g(%d) ... " % n)
  m = 100 // n
  print("The result is %d" % m)
  print("Ending g(%d) ... " % n)

def main():
  while True:
    s = input("Enter a number> ")
    if s.strip() == "":
      return
    try:
      print("Beginning of try block")
      n = int(s)
      f(n)
      print("End of try block")
    except ValueError:
      print("Please enter a number!")
    except ZeroDivisionError:
      print("I can’t handle this value!")

main()

Enter a number> 12
Beginning of try block
Starting f(12) ... 
Starting g(12) ... 
The result is 8
Ending g(12) ... 
Ending f(12) ... 
End of try block
Enter a number> abc
Beginning of try block
Please enter a number!
Enter a number> 0
Beginning of try block
Starting f(0) ... 
Starting g(0) ... 
I can’t handle this value!
Enter a number> 


So far we have only caught exceptions raised inside some library function. But you can just as well raise exceptions yourself. For instance, let’s assume that our function g(n) above should only handle non-negative numbers. We can ensure this by throwing a `ValueError` if the argument is negative. The whole script now looks like this:

In [ ]:
def f(n):
  print("Starting f(%d) ... " % n)
  g(n)
  print("Ending f(%d) ... " % n)

def g(n):
  print("Starting g(%d) ... " % n)
  if n < 0:
    raise ValueError("Cannot handle negative numbers")
  print("The result is %d" % n)
  print("Ending g(%d) ... " % n)

def main():
  while True:
    s = input("Enter a number> ")
    if s.strip() == "":
      return
    try:
      print("Beginning of try block")
      n = int(s)
      f(n)
      print("End of try block")
    except ValueError as e:
      print("Cannot handle this input: %s" % e)

main()

Enter a number> 33
Beginning of try block
Starting f(33) ... 
Starting g(33) ... 
The result is 33
Ending g(33) ... 
Ending f(33) ... 
End of try block


You can also define your own `Exception`

In [ ]:
class MyErrorRandom(Exception):
  def __init__(self):
      super().__init__('custom_error_msg')

def main():
  try:
    x = int(input("I'll raise an error!"))
    raise MyErrorRandom
  except MyErrorRandom as e:
    print('error occured', e)

main()

I'll raise an error!5
error occured custom_error_msg
